In [1]:
import pandas as pd


import string
import math
import re 
from nltk import ngrams

def get_ttr(x):
    tokens = x.split(" ")
    return (len(set(tokens)) / len(tokens)) * 100



def normalize_answer(s):
    """Lower text and remove punctuation and extra whitespace."""


    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))

def modulus(x):
    if x >= 0:
        return x
    elif x < 0: 
        return -(x)



def tokenize(text, nGram=3):
    text = text.split(" ")
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(x, y, nGram):
  overlaps = []
  for h, r in zip(x, y):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, nGram)
    b = tokenize(r, nGram)

    if len(a) >= len(b) and len(a) > 0:
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a) and len(b) > 0:
      overlaps.append(len(set(a) & set(b))/len(b))
    elif len(a) == 0 or len(b) == 0:
      overlaps.append(0)

  return overlaps
       
def read_text(file):
    texts = []
    with open(file) as opned:
        for line in opned:
            texts.append(line.strip())
    return texts






In [174]:

generated = read_text("./models/moderno_arcaico/PT/test.src")

target = read_text("./models/moderno_arcaico/PT/test.tgt")

df_din = pd.DataFrame.from_dict({"source": generated, "target": target})



In [175]:
df_din.iloc[50].source

'Quando Saul e os israelitas ouviram isso, ficaram aterrorizados e muito abalados. Jessé envia Davi ao acampamento de Saul'

In [176]:
df_din.iloc[50].target

'Ouvindo, então, Saul e todo o Israel essas palavras do filisteu, espantaram-se e temeram muito. Jessé envia Davi a seus irmãos'

In [177]:

df_din["trigram"] = getNgramOverlap(df_din["source"].to_list(), df_din["target"].to_list(), 2)


In [165]:
df_din["trigram"]

0       0.275862
1       0.066667
2       0.250000
3       0.423077
4       0.428571
          ...   
8746    0.333333
8747    0.366667
8748    0.153846
8749    0.162791
8750    0.333333
Name: trigram, Length: 8751, dtype: float64

In [178]:
df_filter = df_din[(df_din["trigram"] >= 0.07) & (df_din["trigram"] <= 0.1)]

In [179]:
df_filter

,source,target,trigram
12,Esta é a mensagem contra Edom: Alguém me chama...,"Peso de Dumá. Gritam-me de Seir: Guarda, que h...",0.100000
61,Amem as pessoas sem fingimento. Odeiem tudo qu...,Que o amor que mostrarem pelos outros seja aut...,0.100000
159,Tenho sempre intenso desejo por teus estatutos.,A minha alma está quebrantada de desejar os te...,0.071429
206,"Ao amanhecer ele apareceu novamente no templo,...","Contudo, logo no dia seguinte, de manhã cedo, ...",0.083333
232,— Então o sacerdote fará com que a mulher conc...,E o sacerdote a conjurará e dirá àquela mulher...,0.070175
...,...,...,...
8537,"Antes que os espiões fossem dormir, Raabe foi ...","E, antes que eles dormissem, ela subiu a eles ...",0.076923
8638,"Ainda assim, ele viu a aflição do povo e ouviu...","Contudo, atentou para a sua aflição, ouvindo o...",0.083333
8708,Forjem seus arados para fazer espadas e transf...,Forjai espadas das vossas enxadas e lanças das...,0.100000
8711,"Contudo, só serei misericordioso se vocês aban...","Mas, se deveras melhorardes os vossos caminhos...",0.086957


In [180]:
df_filter.iloc[0].trigram

np.float64(0.1)

In [181]:
df_filter.iloc[0].source

'Esta é a mensagem contra Edom: Alguém me chama do país de Edom e diz: “Guarda, quanto falta para terminar a noite? Guarda, quanto falta para terminar a noite? ”'

In [182]:
df_filter.iloc[0].target

'Peso de Dumá. Gritam-me de Seir: Guarda, que houve de noite? Guarda, que houve de noite?'